In [1]:
!pip install kaggle

!mkdir .kaggle

In [0]:
import json

token = {"username":"anshumitra","key":"04b591f3dd643ef3d92125c815ddb7a6"}

with open('/content/.kaggle/kaggle.json', 'w+') as file:
    json.dump(token, file)

In [5]:
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json

!kaggle config set -n path -v{/content}

!chmod 600 /root/.kaggle/kaggle.json

- path is now set to: {/content}
  0% 0.00/276k [00:00<?, ?B/s]
100% 276k/276k [00:00<00:00, 36.6MB/s]
 46% 12.0M/26.0M [00:00<00:00, 45.9MB/s]
100% 26.0M/26.0M [00:00<00:00, 86.4MB/s]
 71% 9.00M/12.6M [00:00<00:00, 90.3MB/s]
100% 12.6M/12.6M [00:00<00:00, 80.5MB/s]
 69% 9.00M/13.0M [00:00<00:00, 22.8MB/s]
100% 13.0M/13.0M [00:00<00:00, 26.3MB/s]
 79% 20.0M/25.3M [00:00<00:00, 46.7MB/s]
100% 25.3M/25.3M [00:00<00:00, 84.4MB/s]


In [6]:
!kaggle competitions download -c word2vec-nlp-tutorial -p /content

!mkdir movie-data
!mv *.zip sampleSubmission.csv movie-data
%cd movie-data
!ls

!unzip labeledTrainData.tsv.zip
!unzip unlabeledTrainData.tsv.zip
!unzip testData.tsv.zip

!mkdir zip-files
!mv *.zip zip-files/

!ls

/content/movie-data
labeledTrainData.tsv.zip  testData.tsv.zip
sampleSubmission.csv	  unlabeledTrainData.tsv.zip
Archive:  labeledTrainData.tsv.zip
  inflating: labeledTrainData.tsv    
Archive:  unlabeledTrainData.tsv.zip
  inflating: unlabeledTrainData.tsv  
Archive:  testData.tsv.zip
  inflating: testData.tsv            
labeledTrainData.tsv  testData.tsv	      zip-files
sampleSubmission.csv  unlabeledTrainData.tsv


In [15]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews -p /content/movie-data
!unzip imdb-dataset-of-50k-movie-reviews.zip 
!mv *.zip zip-files
!ls

'IMDB Dataset.csv'      sampleSubmission.csv   unlabeledTrainData.tsv
 labeledTrainData.tsv   testData.tsv	       zip-files


In [52]:
!kaggle datasets download -d utathya/imdb-review-dataset -p /content/movie-data
!unzip  imdb-review-dataset.zip
!mv imdb-review-dataset.zip zip-files/
!ls

 finalSubmission.csv   labeledTrainData.tsv   testData.tsv
'IMDB Dataset.csv'     movie-data	      unlabeledTrainData.tsv
 imdb_master.csv       sampleSubmission.csv   zip-files


# Code starts from here

In [17]:
import pandas as pd

df1 = pd.read_csv('labeledTrainData.tsv', delimiter="\t")
df1 = df1.drop(['id'], axis=1)
df1.head()

,sentiment,review
0,1,With all this stuff going down at the moment w...
1,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,0,The film starts with a manager (Nicholas Bell)...
3,0,It must be assumed that those who praised this...
4,1,Superbly trashy and wondrously unpretentious 8...


In [26]:
df2 = pd.read_csv('IMDB Dataset.csv', delimiter=",")
df2['sentiment'] = df2['sentiment'].map({'positive': 1, 'negative': 0})
columnsTitles=["sentiment","review"]
df2=df2.reindex(columns=columnsTitles)
df2.head()


,sentiment,review
0,1,One of the other reviewers has mentioned that ...
1,1,A wonderful little production. <br /><br />The...
2,1,I thought this was a wonderful way to spend ti...
3,0,Basically there's a family where a little boy ...
4,1,"Petter Mattei's ""Love in the Time of Money"" is..."


In [60]:
df3 = pd.read_csv('imdb_master.csv', encoding="latin-1")
df3 = df3.drop(['Unnamed: 0','type','file'],axis=1)
df3.columns = ["review","sentiment"]
df3 = df3[df3.sentiment != 'unsup']
df3['sentiment'] = df3['sentiment'].map({'pos': 1, 'neg': 0})
columnsTitles=["sentiment","review"]
df3=df3.reindex(columns=columnsTitles)
df3.head()

,sentiment,review
0,0,Once again Mr. Costner has dragged out a movie...
1,0,This is an example of why the majority of acti...
2,0,"First of all I hate those moronic rappers, who..."
3,0,Not even the Beatles could write songs everyon...
4,0,Brass pictures (movies is not a fitting word f...


In [61]:
df = pd.concat([df1, df2, df3]).reset_index(drop=True)
print(df.head())
print(len(df))

   sentiment                                             review
0          1  With all this stuff going down at the moment w...
1          1  \The Classic War of the Worlds\" by Timothy Hi...
2          0  The film starts with a manager (Nicholas Bell)...
3          0  It must be assumed that those who praised this...
4          1  Superbly trashy and wondrously unpretentious 8...
125000


In [62]:
import re
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

stop_words = set(stopwords.words("english")) 
lemmatizer = WordNetLemmatizer()


def clean_text(text):
    text = re.sub(r'[^\w\s]','',text, re.UNICODE)
    text = text.lower()
    text = [lemmatizer.lemmatize(token) for token in text.split(" ")]
    text = [lemmatizer.lemmatize(token, "v") for token in text]
    text = [word for word in text if not word in stop_words]
    text = " ".join(text)
    return text

df['Processed_Reviews'] = df.review.apply(lambda x: clean_text(x))

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [67]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense , Input , LSTM , Embedding, Dropout , Activation, GRU, Flatten
from keras.layers import Bidirectional, GlobalMaxPool1D, MaxPooling1D
from keras.models import Model, Sequential
from keras.layers import Convolution1D
from keras import initializers, regularizers, constraints, optimizers, layers

max_features = 6000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(df['Processed_Reviews'])
list_tokenized_train = tokenizer.texts_to_sequences(df['Processed_Reviews'])

maxlen = 130
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
y = df['sentiment']

embed_size = 128
model = Sequential()
model.add(Embedding(max_features, embed_size))
model.add(Convolution1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(100))
model.add(Dense(20, activation="relu"))
model.add(Dropout(0.05))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

batch_size = 100
epochs = 5
model.fit(X_t,y, batch_size=batch_size, epochs=epochs, validation_split=0.2)

Train on 100000 samples, validate on 25000 samples
Epoch 1/5
100000/100000 [==============================] - 93s 929us/step - loss: 0.2758 - acc: 0.8871 - val_loss: 0.1642 - val_acc: 0.9448
Epoch 2/5
100000/100000 [==============================] - 88s 885us/step - loss: 0.1476 - acc: 0.9477 - val_loss: 0.0693 - val_acc: 0.9802
Epoch 3/5
100000/100000 [==============================] - 88s 875us/step - loss: 0.0652 - acc: 0.9787 - val_loss: 0.0323 - val_acc: 0.9896
Epoch 4/5
 36400/100000 [=========>....................] - ETA: 55s - loss: 0.0260 - acc: 0.9920Train on 100000 samples, validate on 25000 samples
Epoch 1/5
100000/100000 [==============================] - 93s 929us/step - loss: 0.2758 - acc: 0.8871 - val_loss: 0.1642 - val_acc: 0.9448
Epoch 2/5
100000/100000 [==============================] - 88s 885us/step - loss: 0.1476 - acc: 0.9477 - val_loss: 0.0693 - val_acc: 0.9802
Epoch 3/5
100000/100000 [==============================] - 88s 875us/step - loss: 0.0652 - acc: 0.9787

"\n\nembedding_vecor_length = 32\n\nmodel = Sequential()\nmodel.add(Embedding(19479, embedding_vecor_length, input_length=max_review_length, dropout=0.2))\nmodel.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))\nmodel.add(MaxPooling1D(pool_size=2))\n# 1 layer of 100 units in the hidden layers of the LSTM cells\nmodel.add(LSTM(100))\nmodel.add(Dense(5, activation='softmax'))\nmodel.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])\nprint(model.summary())\nmodel.fit(X_train, y_train,validation_split=0.20, epochs=5,verbose=1, batch_size=32,callbacks=cbks)\n"

"\n\nembedding_vecor_length = 32\n\nmodel = Sequential()\nmodel.add(Embedding(19479, embedding_vecor_length, input_length=max_review_length, dropout=0.2))\nmodel.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))\nmodel.add(MaxPooling1D(pool_size=2))\n# 1 layer of 100 units in the hidden layers of the LSTM cells\nmodel.add(LSTM(100))\nmodel.add(Dense(5, activation='softmax'))\nmodel.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])\nprint(model.summary())\nmodel.fit(X_train, y_train,validation_split=0.20, epochs=5,verbose=1, batch_size=32,callbacks=cbks)\n"

In [68]:
df_test=pd.read_csv("testData.tsv",header=0, delimiter="\t", quoting=3)
df_test.head()
df_test["review"]=df_test.review.apply(lambda x: clean_text(x))
df_test["sentiment"] = df_test["id"].map(lambda x: 1 if int(x.strip('"').split("_")[1]) >= 5 else 0)
y_test = df_test["sentiment"]
list_sentences_test = df_test["review"]
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_te = pad_sequences(list_tokenized_test, maxlen=maxlen)
prediction = model.predict(X_te)
y_pred = (prediction > 0.5)
from sklearn.metrics import f1_score, confusion_matrix
print('F1-score: {0}'.format(f1_score(y_pred, y_test)))
print('Confusion matrix:')
confusion_matrix(y_pred, y_test)

F1-score: 0.9966748127078242
Confusion matrix:


array([[12478,    61],
       [   22, 12439]])

In [32]:
print(df_test.head())


           id                                             review  sentiment
0  "12311_10"  naturally film main theme mortality nostalgia ...          1
1    "8348_2"  movie disaster within disaster film full great...          0
2    "5828_4"  movie kid saw tonight child love one point kid...          0
3    "7186_2"  afraid dark leave impression several different...          0
4   "12128_7"  accurate depiction small time mob life film ne...          1


In [69]:
import numpy as np

submission = pd.DataFrame([df_test.id]).transpose()
#submission.drop(submission.tail(1).index,inplace=True)

y_pred = y_pred.astype(int)
print(len(submission))
print(len(y_pred))

submission['sentiment'] = y_pred.flatten()
submission['id'] = submission.id.str.replace('"', '')
submission.head()

25000
25000


,id,sentiment
0,12311_10,1
1,8348_2,0
2,5828_4,0
3,7186_2,0
4,12128_7,1


In [0]:
submission.to_csv('finalSubmission.csv', index=False)

In [0]:
!ls

finalSubmission.csv   sampleSubmission.csv  unlabeledTrainData.tsv
labeledTrainData.tsv  testData.tsv	    zip-files


In [0]:
from google.colab import files
files.download('/content/movie-data/finalSubmission.csv') 